In [1]:
import numpy as np
import pandas as pd

In [2]:
ahmd_traffic = pd.read_excel("data/Events Report of Drivers in Ahmedabad_16-07 to 31-07.xlsx")

In [3]:
ahmd_traffic.head()

,Event Name,Event Time,Event Value (Upper limit =51km/h),Event Unit,Bus no.,Route Name,Stop Code,Stop Name,Latitude,Longitude
0,Idling,16.07.2018 03:05:47,180.0,Seconds,31,5U_Vasna - Naroda Gam_NORMAL,275,Vasna,23.002243,72.547849
1,Idling,16.07.2018 03:08:12,180.0,Seconds,34,6E_Narol - Odhav Talav_NORMAL,77,Narol,22.972980,72.589567
2,Idling,16.07.2018 03:08:47,180.0,Seconds,31,5U_Vasna - Naroda Gam_NORMAL,275,Vasna,23.002243,72.547849
3,Idling,16.07.2018 03:11:12,180.0,Seconds,34,6E_Narol - Odhav Talav_NORMAL,77,Narol,22.972980,72.589567
4,Idling,16.07.2018 03:12:26,180.0,Seconds,31,5U_Vasna - Naroda Gam_NORMAL,275,Vasna,23.002243,72.547849


#### Droping "Event Value" and "Event Unit" as there would be information leakage for predicting "Event Name" eg, if event units is seconds only idling could be the case. Also getting rid of other less useful columns

In [4]:
ahmd_traffic = ahmd_traffic.drop(ahmd_traffic.columns[[2,3,6,8,9]], axis = 1)
ahmd_traffic.head()

,Event Name,Event Time,Bus no.,Route Name,Stop Name
0,Idling,16.07.2018 03:05:47,31,5U_Vasna - Naroda Gam_NORMAL,Vasna
1,Idling,16.07.2018 03:08:12,34,6E_Narol - Odhav Talav_NORMAL,Narol
2,Idling,16.07.2018 03:08:47,31,5U_Vasna - Naroda Gam_NORMAL,Vasna
3,Idling,16.07.2018 03:11:12,34,6E_Narol - Odhav Talav_NORMAL,Narol
4,Idling,16.07.2018 03:12:26,31,5U_Vasna - Naroda Gam_NORMAL,Vasna


#### Creating new variables, weekday and hour of the day based on timestamp column, "Event Time"

In [5]:
ahmd_traffic['Event Time'] =  pd.to_datetime(ahmd_traffic['Event Time'], format='%d.%m.%Y %H:%M:%S')
ahmd_traffic['weekday'] = ahmd_traffic['Event Time'].dt.dayofweek
ahmd_traffic['hour'] = ahmd_traffic['Event Time'].dt.hour
ahmd_traffic = ahmd_traffic.drop(ahmd_traffic.columns[[1]], axis = 1)

days = {0:'Mon',1:'Tues',2:'Weds',3:'Thurs',4:'Fri',5:'Sat',6:'Sun'}

ahmd_traffic['weekday'] = ahmd_traffic['weekday'].apply(lambda x: days[x])
ahmd_traffic.head()

,Event Name,Bus no.,Route Name,Stop Name,weekday,hour
0,Idling,31,5U_Vasna - Naroda Gam_NORMAL,Vasna,Mon,3
1,Idling,34,6E_Narol - Odhav Talav_NORMAL,Narol,Mon,3
2,Idling,31,5U_Vasna - Naroda Gam_NORMAL,Vasna,Mon,3
3,Idling,34,6E_Narol - Odhav Talav_NORMAL,Narol,Mon,3
4,Idling,31,5U_Vasna - Naroda Gam_NORMAL,Vasna,Mon,3


#### Converting categorial variables into integers (label encoding before embedding operation)

In [6]:
from sklearn import preprocessing # Label Encoding: converting text labels into the integers
from sklearn.externals import joblib #for saving label encoder, we will need it later

event_name_le = preprocessing.LabelEncoder()
route_name_le = preprocessing.LabelEncoder()
stop_name_le = preprocessing.LabelEncoder()
weekday_le = preprocessing.LabelEncoder()


event_name_le.fit(ahmd_traffic['Event Name'])
ahmd_traffic['Event Name'] = event_name_le.transform(ahmd_traffic['Event Name'])

route_name_le.fit(ahmd_traffic['Route Name'])
ahmd_traffic['Route Name'] = route_name_le.transform(ahmd_traffic['Route Name'])

stop_name_le.fit(ahmd_traffic['Stop Name'])
ahmd_traffic['Stop Name'] = stop_name_le.transform(ahmd_traffic['Stop Name'])

weekday_le.fit(ahmd_traffic['weekday'])
ahmd_traffic['weekday'] = weekday_le.transform(ahmd_traffic['weekday'])

joblib.dump(event_name_le, 'data/label_encoders/event_name_le.pkl')
joblib.dump(route_name_le, 'data/label_encoders/route_name_le.pkl')
joblib.dump(stop_name_le, 'data/label_encoders/stop_name_le.pkl')
joblib.dump(weekday_le, 'data/label_encoders/weekday_le.pkl')

['data/label_encoders/weekday_le.pkl']

#### Saving processed dataframe

In [7]:
ahmd_traffic.to_csv("data/ahmd_traffic_clean_df.csv", index = False)
ahmd_traffic.head()

,Event Name,Bus no.,Route Name,Stop Name,weekday,hour
0,2,31,83,176,1,3
1,2,34,86,105,1,3
2,2,31,83,176,1,3
3,2,34,86,105,1,3
4,2,31,83,176,1,3
